### CNNでより高精度なCIFAR10の分類器を実装

### ルール

- 訓練データはx_train， t_train，テストデータはx_testで与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されているx_train，t_train以外の学習データは使わないでください．**
- 今回から基本的にAPI制限はありません．
- ただしCNNベースでないモデル（Vision Transformerなど）やtorchvision等の既存モデル，学習済みモデルは用いないでください．

### データの読み込み

In [3]:
import random

import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

#学習データ
x_train = np.load('data/x_train.npy')
t_train = np.load('data/t_train.npy')
    
#テストデータ
x_test = np.load('data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        data = x_train.astype('float32')
        self.x_train = []
        for i in range(data.shape[0]):
            self.x_train.append(Image.fromarray(np.uint8(data[i])))
        self.t_train = t_train
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        return self.transform(self.x_train[idx]), torch.tensor(t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        data = x_test.astype('float32')
        self.x_test = []
        for i in range(data.shape[0]):
            self.x_test.append(Image.fromarray(np.uint8(data[i])))
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.x_test)

    def __getitem__(self, idx):
        return self.transform(self.x_test[idx])

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 畳み込みニューラルネットワーク(CNN)の実装

In [4]:
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


fix_seed(seed=42)


class gcn():
    def __init__(self):
        pass

    def __call__(self, x):
        mean = torch.mean(x)
        std = torch.std(x)
        return (x - mean)/(std + 10**(-6))  # 0除算を防ぐ


class ZCAWhitening():
    def __init__(self, epsilon=1e-4, device="cuda"):  # 計算が重いのでGPUを用いる
        self.epsilon = epsilon
        self.device = device

    def fit(self, images):  # 変換行列と平均をデータから計算
        x = images[0][0].reshape(1, -1)
        self.mean = torch.zeros([1, x.size()[1]]).to(self.device)
        con_matrix = torch.zeros([x.size()[1], x.size()[1]]).to(self.device)
        for i in range(len(images)):  # 各データについての平均を取る
            x = images[i][0].reshape(1, -1).to(self.device)
            self.mean += x / len(images)
            con_matrix += torch.mm(x.t(), x) / len(images)
            if i % 10000 == 0:
                print("{0}/{1}".format(i, len(images)))
        self.E, self.V = torch.linalg.eigh(con_matrix)  # 固有値分解
        self.E = torch.max(self.E, torch.zeros_like(self.E)) # 誤差の影響で負になるのを防ぐ
        self.ZCA_matrix = torch.mm(torch.mm(self.V, torch.diag((self.E.squeeze()+self.epsilon)**(-0.5))), self.V.t())
        print("completed!")

    def __call__(self, x):
        size = x.size()
        x = x.reshape(1, -1).to(self.device)
        x -= self.mean
        x = torch.mm(x, self.ZCA_matrix.t())
        x = x.reshape(tuple(size))
        x = x.to("cpu")
        return x


# (datasetのクラスを自作したので，このあたりの処理が少し変わっています)

zca = ZCAWhitening()
zca.fit(trainval_data)

val_size = 3000

# 訓練データと検証データに分割
train_data, val_data = torch.utils.data.random_split(trainval_data, [len(trainval_data)-val_size, val_size])  


# 前処理を定義
transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.RandomHorizontalFlip(),
                                       zca])

transform = transforms.Compose([transforms.ToTensor(),
                                zca])

# データセットに前処理を設定
trainval_data.transform = transform_train
test_data.transform = transform

batch_size = 64

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)


0/50000
10000/50000
20000/50000
30000/50000
40000/50000
completed!


In [5]:
import torch.nn as nn
import torch.optim as optim
# import torch.autograd as autograd
# import torch.nn.functional as F

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


conv_net = nn.Sequential(
    nn.Conv2d(3, 32, 3, padding=1),     # 32x32x3 -> 32x32x32
    nn.BatchNorm2d(32),
    nn.GELU(),
    nn.Conv2d(32, 32, 3, padding=1),    # 32x32x32 -> 32x32x32
    nn.BatchNorm2d(32),
    nn.GELU(),
    nn.MaxPool2d(2),                     # 32x32x32 -> 16x16x32

    nn.Conv2d(32, 64, 3, padding=1),    # 16x16x32 -> 16x16x64
    nn.BatchNorm2d(64),
    nn.GELU(),
    nn.Conv2d(64, 64, 3, padding=1),    # 16x16x64 -> 16x16x64
    nn.BatchNorm2d(64),
    nn.GELU(),
    nn.MaxPool2d(2),                     # 16x16x64 -> 8x8x64

    nn.Conv2d(64, 128, 3, padding=1),   # 8x8x64 -> 8x8x128
    nn.BatchNorm2d(128),
    nn.GELU(),
    nn.Conv2d(128, 128, 3, padding=1),  # 8x8x128 -> 8x8x128
    nn.BatchNorm2d(128),
    nn.GELU(),
    nn.MaxPool2d(2),                     # 8x8x128 -> 4x4x128

    nn.Flatten(),
    nn.Linear(4*4*128, 512),
    nn.GELU(),
    nn.Dropout(p=0.5),
    nn.Linear(512, 256),
    nn.GELU(),
    nn.Dropout(p=0.5),
    nn.Linear(256, 10)
)

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)

conv_net.apply(init_weights)


n_epochs = 200
lr = 0.001
device = 'cuda'

conv_net.to(device)
optimizer = optim.Adam(conv_net.parameters(), lr=lr) # WRITE ME
loss_function = nn.CrossEntropyLoss() # WRITE ME

In [13]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:

        # WRITE ME
        n_train += t.size()[0]

        conv_net.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = conv_net.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())
        # WRITE ME

    conv_net.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        # WRITE ME
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = conv_net.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
        # WRITE ME

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))


EPOCH: 0, Train [Loss: 1.443, Accuracy: 0.467], Valid [Loss: 1.045, Accuracy: 0.643]
EPOCH: 1, Train [Loss: 1.018, Accuracy: 0.646], Valid [Loss: 0.816, Accuracy: 0.719]
EPOCH: 2, Train [Loss: 0.805, Accuracy: 0.725], Valid [Loss: 0.676, Accuracy: 0.760]
EPOCH: 3, Train [Loss: 0.677, Accuracy: 0.771], Valid [Loss: 0.678, Accuracy: 0.767]
EPOCH: 4, Train [Loss: 0.593, Accuracy: 0.801], Valid [Loss: 0.577, Accuracy: 0.797]
EPOCH: 5, Train [Loss: 0.524, Accuracy: 0.823], Valid [Loss: 0.577, Accuracy: 0.805]
EPOCH: 6, Train [Loss: 0.472, Accuracy: 0.840], Valid [Loss: 0.502, Accuracy: 0.821]
EPOCH: 7, Train [Loss: 0.429, Accuracy: 0.855], Valid [Loss: 0.493, Accuracy: 0.831]
EPOCH: 8, Train [Loss: 0.392, Accuracy: 0.867], Valid [Loss: 0.519, Accuracy: 0.819]
EPOCH: 9, Train [Loss: 0.356, Accuracy: 0.880], Valid [Loss: 0.478, Accuracy: 0.835]
EPOCH: 10, Train [Loss: 0.327, Accuracy: 0.888], Valid [Loss: 0.513, Accuracy: 0.832]
EPOCH: 11, Train [Loss: 0.302, Accuracy: 0.900], Valid [Loss: 0.

In [9]:
conv_net.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = conv_net.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture05/submission_pred.csv', header=True, index_label='id')

KeyboardInterrupt: 